# 定义函数，OpenAI 会根据问题自动选择合适的函数

In [1]:
function_descriptions = [
    {
        "name": "create_order",
        "description": "用给定的参数创建订单",
        "parameters": {
            "type": "object",
            "properties": {
                "orders": {
                    "type": "string",
                    "description": "识别的待创建订单的参数。参数的结构为 [{'receiverName': '', 'receiverMobile': ''}]",
                },
            },
            "required": ["orders"],
        },
    }
]

In [2]:
user_content = "给刘德华（电话13400000000）送一斤吧。还有个叫张学友，电话13500000000也送。好了，下单吧。"
user_query = f"""请根据给出的信息抽取相应的参数创建订单
1. 通过命名实体识别先对输入的信息抽取特定的参数。抽取的数据结构如下：
[{{"receiverName": "", "receiverMobile": ""}}]
2. 请用命名实体识别抽取的数组参数，不做任何修改使用创建订单接口创建订单

输入信息：
'''
{user_content}
'''
"""

In [3]:
import openai
# 运行此API配置，需要将目录中的.env中api_key替换为自己的
from dotenv import load_dotenv, find_dotenv
import json
import os

# read local .env file
load_dotenv(find_dotenv())

openai.api_key = os.environ['OPENAI_API_KEY']
model = 'gpt-3.5-turbo-16k-0613'
response = openai.ChatCompletion.create(
    model=model,
    messages=[{"role": "user", "content": user_query}],
    functions=function_descriptions,
    function_call="auto"
)
ai_response_message = response["choices"][0]["message"]
dumps = json.dumps(ai_response_message, ensure_ascii=False, indent=2)
ai_response_message = json.loads(dumps)
print(ai_response_message)

{'role': 'assistant', 'content': None, 'function_call': {'name': 'create_order', 'arguments': '{\n  "orders": "[{\\"receiverName\\": \\"刘德华\\", \\"receiverMobile\\": \\"13400000000\\"}, {\\"receiverName\\": \\"张学友\\", \\"receiverMobile\\": \\"13500000000\\"}]"\n}'}}


# 根据 function call 返回的函数名及参数，调用函数

In [4]:
import inspect

def get_function_parameter_names(function):
  if function is not None and inspect.isfunction(function):
      parameter_names = inspect.signature(function).parameters.keys()
      return list(parameter_names)
  else:
      return None

In [5]:
def create_order(orders):
    """用给定的参数创建订单 参数为 [{'receiverName': '', 'receiverMobile': ''}]"""
    # orders 为字符串，需要转换为 list 对象
    orders = json.loads(orders)
    for order_info in orders:
        receiver_name = order_info["receiverName"]
        receiver_mobile = order_info["receiverMobile"]
        print(f'create order for  {receiver_name}/{receiver_mobile}\n')
    return f"""{orders}\n订单创建成功!!!!!!!!"""


In [6]:
function_name = ai_response_message["function_call"]["name"]
arguments = json.loads(ai_response_message["function_call"]["arguments"])

# Locate the function and make the call
def cal_function(_function_name, _arguments):
    the_function = globals().get(_function_name)
    parameter_names = get_function_parameter_names(the_function)
    parameter_values = []
    for parameter_name in parameter_names:
      parameter_values.append(_arguments[parameter_name])

    return the_function(*parameter_values)

returned_value = cal_function(function_name, arguments)
print(returned_value)

create order for  刘德华/13400000000

create order for  张学友/13500000000

[{'receiverName': '刘德华', 'receiverMobile': '13400000000'}, {'receiverName': '张学友', 'receiverMobile': '13500000000'}]
订单创建成功!!!!!!!!


# 根据返回的函数调用结果，让 OpenAI 总结任务结果

In [7]:
second_response = openai.ChatCompletion.create(
    model=model,
    messages=[
        {"role": "user", "content": user_query},
        ai_response_message,
        {
            "role": "function",
            "name": "crete_order",
            "content": returned_value
        },
    ],
    temperature=0,
)
print('任务：\n', user_query, '\n')
print ('结果：\n', second_response['choices'][0]['message']['content'])

任务：
 请根据给出的信息抽取相应的参数创建订单
1. 通过命名实体识别先对输入的信息抽取特定的参数。抽取的数据结构如下：
[{"receiverName": "", "receiverMobile": ""}]
2. 请用命名实体识别抽取的数组参数，不做任何修改使用创建订单接口创建订单

输入信息：
'''
给刘德华（电话13400000000）送一斤吧。还有个叫张学友，电话13500000000也送。好了，下单吧。
'''
 

结果：
 根据输入的信息，抽取到的参数为：
- 收件人姓名：刘德华
- 收件人电话：13400000000
- 收件人姓名：张学友
- 收件人电话：13500000000

使用这些参数创建订单成功。
